<a href="https://colab.research.google.com/github/gazuty/betfair-dashboard/blob/main/Results.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [40]:
# --- CONFIGURATION ---

import os
from datetime import datetime

# ─── Your Drive / folder paths ───
BASE_FOLDER       = '/content/drive/My Drive/Betfair'
MASTER_CSV        = os.path.join(BASE_FOLDER, 'Betfair_Master.csv')
ARCHIVE_FOLDER    = os.path.join(BASE_FOLDER, 'Archive')
BETTING_PATTERN   = os.path.join(BASE_FOLDER, 'BettingPandL*.csv')

# ─── Google Sheet settings ───
GOOGLE_SHEET_NAME = 'Betfair Dashboard'

# ─── Business rules ───
VALID_SPORTS      = ['Horse Racing', 'Greyhound Racing']
MIN_STRIKE_BETS   = 50

# ─── Setup ───
os.makedirs(ARCHIVE_FOLDER, exist_ok=True)

print("✅ Configuration loaded:")
print(f"  BASE_FOLDER        = {BASE_FOLDER}")
print(f"  MASTER_CSV         = {MASTER_CSV}")
print(f"  ARCHIVE_FOLDER     = {ARCHIVE_FOLDER}")
print(f"  BETTING_PATTERN    = {BETTING_PATTERN}")
print(f"  GOOGLE_SHEET_NAME  = {GOOGLE_SHEET_NAME}")
print(f"  VALID_SPORTS       = {VALID_SPORTS}")
print(f"  MIN_STRIKE_BETS    = {MIN_STRIKE_BETS}")


✅ Configuration loaded:
  BASE_FOLDER        = /content/drive/My Drive/Betfair
  MASTER_CSV         = /content/drive/My Drive/Betfair/Betfair_Master.csv
  ARCHIVE_FOLDER     = /content/drive/My Drive/Betfair/Archive
  BETTING_PATTERN    = /content/drive/My Drive/Betfair/BettingPandL*.csv
  GOOGLE_SHEET_NAME  = Betfair Dashboard
  VALID_SPORTS       = ['Horse Racing', 'Greyhound Racing']
  MIN_STRIKE_BETS    = 50


In [41]:
# --- STEP 1: Master Updater ---

import pandas as pd, glob, os, shutil

REQUIRED_COLS = ['Market', 'Settled date']

def update_betfair_master():
    print("🔄 Starting master update")

    # 1️⃣ Load or initialize master
    if os.path.exists(MASTER_CSV):
        df_master = pd.read_csv(MASTER_CSV)
        df_master['Settled date'] = pd.to_datetime(df_master['Settled date'], errors='coerce')
        df_master['Profit_Loss'] = pd.to_numeric(df_master['Profit_Loss'], errors='coerce')
        df_master = df_master.dropna(subset=['Settled date']).reset_index(drop=True)
        print(f"✅ Loaded master ({len(df_master)} rows)")
    else:
        print("⚠ No existing master found — starting fresh")
        df_master = pd.DataFrame(columns=REQUIRED_COLS + ['Profit_Loss'])

    # 2️⃣ Gather raw files
    raw_files = glob.glob(BETTING_PATTERN)
    print(f"📂 Found {len(raw_files)} raw file(s)")

    if not raw_files:
        print("⚠ No raw files to process — exiting.")
        return

    # 3️⃣ Process each file
    dfs = []
    for filepath in raw_files:
        fname = os.path.basename(filepath)
        print(f"📥 {fname}", end="")

        df = pd.read_csv(filepath)
        missing = [c for c in REQUIRED_COLS if c not in df.columns]
        if missing:
            print(f" → ❌ missing columns {missing}")
            continue

        profs = [c for c in df.columns if 'profit' in c.lower()]
        if not profs:
            print(" → ❌ no profit column found")
            continue

        pick = next((c for c in profs if 'aud' in c.lower()), profs[0])
        df['Profit_Loss'] = pd.to_numeric(df[pick], errors='coerce')
        df['Settled date'] = pd.to_datetime(df['Settled date'], errors='coerce')
        df = df[['Market', 'Settled date', 'Profit_Loss']].dropna(subset=['Settled date'])

        dfs.append(df)
        print(f" → {len(df)} rows from '{pick}'")

    if not dfs:
        print("⚠ No valid data loaded from raw files — exiting.")
        return

    # 4️⃣ Combine and deduplicate
    df_new = pd.concat(dfs, ignore_index=True)

    # Drop invalid dates (again, to be safe)
    df_new = df_new.dropna(subset=['Settled date']).reset_index(drop=True)

    # Prepare deduplication keys
    df_master['_key'] = (
        df_master['Market'].astype(str) + "|" +
        df_master['Settled date'].dt.strftime('%Y-%m-%d %H:%M:%S') + "|" +
        df_master['Profit_Loss'].astype(str)
    )
    df_new['_key'] = (
        df_new['Market'].astype(str) + "|" +
        df_new['Settled date'].dt.strftime('%Y-%m-%d %H:%M:%S') + "|" +
        df_new['Profit_Loss'].astype(str)
    )

    df_unique = df_new[~df_new['_key'].isin(df_master['_key'])]
    print(f"✅ {len(df_unique)} unique new row(s) identified")

    # 5️⃣ Merge and save
    if not df_unique.empty:
        df_combined = pd.concat([
            df_master.drop(columns=['_key']),
            df_unique.drop(columns=['_key'])
        ], ignore_index=True)
        df_combined.to_csv(MASTER_CSV, index=False)
        print(f"✅ Master updated ({len(df_combined)} rows) → {MASTER_CSV}")
    else:
        print("⚠ No new rows to add — master unchanged.")

    # 6️⃣ Archive files
    for filepath in raw_files:
        fname = os.path.basename(filepath)
        shutil.move(filepath, os.path.join(ARCHIVE_FOLDER, fname))
        print(f"📦 Archived {fname}")

# Run the function
update_betfair_master()


🔄 Starting master update
✅ Loaded master (18911 rows)
📂 Found 0 raw file(s)
⚠ No raw files to process — exiting.


In [42]:
# --- STEP 2: Load Master ---

import pandas as pd

print(f"📂 Loading master from: {MASTER_CSV}")
df = pd.read_csv(MASTER_CSV)

# Ensure correct dtypes
df['Settled date'] = pd.to_datetime(df['Settled date'], errors='coerce')
df['Profit_Loss'] = pd.to_numeric(df['Profit_Loss'], errors='coerce')

# Drop rows with invalid dates
before = len(df)
df = df.dropna(subset=['Settled date']).reset_index(drop=True)
after = len(df)

print(f"✅ {after} rows loaded (dropped {before - after} invalid dates).")
print(f"   Profit_Loss dtype: {df['Profit_Loss'].dtype}")


📂 Loading master from: /content/drive/My Drive/Betfair/Betfair_Master.csv
✅ 18911 rows loaded (dropped 0 invalid dates).
   Profit_Loss dtype: float64


In [43]:
# --- STEP 3: Feature Extraction ---

# 1️⃣ Extract Sport from Market (first token before slash)
df['Sport'] = df['Market'].str.extract(r'^([^/]+)/')[0].str.strip()

# 2️⃣ Extract Track_Info and Event_Description for racing sports
racing_mask = df['Sport'].isin(VALID_SPORTS)
track_event = df.loc[racing_mask, 'Market'].str.extract(r'/\s*(.*?)\s*:\s*(.*)')
track_event.columns = ['Track_Info', 'Event_Description']
df.loc[racing_mask, ['Track_Info', 'Event_Description']] = track_event

# 3️⃣ Extract Country from parentheses in Track_Info
df['Country'] = df['Track_Info'].str.extract(r'\(([^)]+)\)')[0]

# 4️⃣ Fill missing country values
df['Country'] = df['Country'].fillna('UK')  # Default to UK for missing codes
df.loc[~df['Sport'].isin(VALID_SPORTS), 'Country'] = 'Unknown'  # Unknown for non-racing

# 5️⃣ Clean up Track_Info to produce Track_Name (remove dates and country)
df['Track_Name'] = (
    df['Track_Info']
      .str.replace(r'\([^)]*\)', '', regex=True)  # Remove (AUS), (US), etc.
      .str.replace(r'\b\d{1,2}(?:st|nd|rd|th)?\s+\w+\b', '', regex=True)  # Remove date-like tokens
      .str.strip()
)

# 6️⃣ Preview output
preview = df.loc[df['Track_Name'].notna(), ['Sport', 'Track_Name', 'Country']].drop_duplicates().head(10)
print("✅ Feature extraction complete — first few tracks:")
print(preview)


✅ Feature extraction complete — first few tracks:
           Sport            Track_Name Country
2   Horse Racing              Ballarat     AUS
3   Horse Racing             Casterton     AUS
5   Horse Racing          Charles Town      US
6   Horse Racing       Canterbury Park      US
7   Horse Racing      Evangeline Downs      US
8   Horse Racing       Churchill Downs      US
10  Horse Racing  Belmont At The Big A      US
11  Horse Racing              Woodbine      US
13  Horse Racing         Monmouth Park      US
14  Horse Racing             Doncaster      UK


In [44]:
# --- STEP 4: Build summary tables (daily, weekly, monthly, sport, country) ---

# 1️⃣ By Day (chronological daily summary with cumulative P/L)
by_day = (
    df.groupby(df['Settled date'].dt.date)['Profit_Loss']
      .sum()
      .reset_index(name='Profit_Loss')
      .rename(columns={'Settled date': 'Day'})
)
by_day = by_day.sort_values('Day').reset_index(drop=True)
by_day['Cumulative_Profit_Loss'] = by_day['Profit_Loss'].cumsum()
by_day[['Profit_Loss', 'Cumulative_Profit_Loss']] = by_day[['Profit_Loss', 'Cumulative_Profit_Loss']].round(2)

# 2️⃣ By Week (starts Mondays)
by_week = (
    df.set_index('Settled date')
      .resample('W-MON')['Profit_Loss']
      .sum()
      .reset_index()
      .rename(columns={'Settled date': 'Week Starting'})
)
by_week['Profit_Loss'] = by_week['Profit_Loss'].round(2)

# 3️⃣ By Month
by_month = (
    df.set_index('Settled date')
      .resample('M')['Profit_Loss']
      .sum()
      .reset_index()
)
by_month['Month'] = by_month['Settled date'].dt.to_period('M').astype(str)
by_month = by_month[['Month', 'Profit_Loss']]
by_month['Profit_Loss'] = by_month['Profit_Loss'].round(2)

# 4️⃣ By Sport
by_sport = (
    df.groupby('Sport')['Profit_Loss']
      .sum()
      .reset_index()
      .round({'Profit_Loss': 2})
)

# 5️⃣ By Country
by_country = (
    df.groupby('Country')['Profit_Loss']
      .sum()
      .reset_index()
      .round({'Profit_Loss': 2})
)

# 6️⃣ Daily summaries for each sport (with cumulative P/L)
sport_daily = {}
for sport in df['Sport'].dropna().unique():
    temp = (
        df[df['Sport'] == sport]
          .groupby(df['Settled date'].dt.date)['Profit_Loss']
          .sum()
          .reset_index(name='Profit_Loss')
          .rename(columns={'Settled date': 'Day'})
          .sort_values('Day')
          .reset_index(drop=True)
    )
    temp['Cumulative_Profit_Loss'] = temp['Profit_Loss'].cumsum().round(2)
    temp['Profit_Loss'] = temp['Profit_Loss'].round(2)
    sport_daily[f"{sport} Daily"] = temp

# 7️⃣ Final summary checks
print(f"✅ By Day: {len(by_day)} rows (last: {by_day['Day'].max()})")
print(f"✅ By Week: {len(by_week)} rows (last: {by_week['Week Starting'].max().date()})")
print(f"✅ By Month: {len(by_month)} rows (last: {by_month['Month'].max()})")
print(f"✅ By Sport: {len(by_sport)} sports → {by_sport['Sport'].tolist()}")
print(f"✅ By Country: {len(by_country)} countries → {by_country['Country'].tolist()}")


✅ By Day: 193 rows (last: 2025-07-12)
✅ By Week: 28 rows (last: 2025-07-14)
✅ By Month: 7 rows (last: 2025-07)
✅ By Sport: 14 sports → ['American Football', 'Basketball', 'Cricket', 'Darts', 'Football', 'Golf', 'Greyhound Racing', 'Horse Racing', 'Ice Hockey', 'Motor Sport', 'Politics', 'Rugby Union', 'Snooker', 'Tennis']
✅ By Country: 8 countries → ['AUS', 'FRA', 'NZL', 'RSA', 'UAE', 'UK', 'US', 'Unknown']


/tmp/ipython-input-44-119278825.py:27: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  .resample('M')['Profit_Loss']


In [45]:
# --- STEP 5: Track Summaries ---

# 1️⃣ Aggregate P/L per track for Horse and Greyhound Racing
track_df = (
    df[df['Sport'].isin(VALID_SPORTS)]
      .groupby(['Sport', 'Track_Name'], as_index=False)['Profit_Loss']
      .sum()
)
track_df['Profit_Loss'] = track_df['Profit_Loss'].round(2)

# 2️⃣ Create summary groups
tracks = {
    'Track Stats':               track_df,
    'Top Horse Tracks':          track_df.query("Sport == 'Horse Racing'").nlargest(15, 'Profit_Loss'),
    'Bottom Horse Tracks':       track_df.query("Sport == 'Horse Racing'").nsmallest(15, 'Profit_Loss'),
    'Top Greyhound Tracks':      track_df.query("Sport == 'Greyhound Racing'").nlargest(15, 'Profit_Loss'),
    'Bottom Greyhound Tracks':   track_df.query("Sport == 'Greyhound Racing'").nsmallest(15, 'Profit_Loss'),
}

# 3️⃣ Preview sample
print("✅ Track summaries built.")
print(" • Sample Track Stats:")
print(track_df.head())
print(" • Top Horse Tracks:")
print(tracks['Top Horse Tracks'][['Track_Name', 'Profit_Loss']].head())


✅ Track summaries built.
 • Sample Track Stats:
              Sport   Track_Name  Profit_Loss
0  Greyhound Racing  Albion Park        91.77
1  Greyhound Racing   Angle Park      -128.51
2  Greyhound Racing     Ballarat        84.36
3  Greyhound Racing      Bendigo        24.32
4  Greyhound Racing  Broken Hill        52.63
 • Top Horse Tracks:
    Track_Name  Profit_Loss
169  Geraldton      1400.78
57     Aintree      1286.07
297   Rosehill      1198.06
255  Newcastle      1030.43
315  Southwell      1026.10


In [46]:
# --- STEP 6: Strike Rates ---

# 1️⃣ Filter to Horse & Greyhound Racing
df_racing = df[df['Sport'].isin(VALID_SPORTS)].copy()

# 2️⃣ Compute total bets and wins per track
strike_df = (
    df_racing
      .groupby(['Sport', 'Track_Name'])['Profit_Loss']
      .agg(
          total_bets='count',
          wins=lambda x: (x > 0).sum()
      )
      .reset_index()
)

# 3️⃣ Calculate strike rate
strike_df['Strike_Rate'] = (strike_df['wins'] / strike_df['total_bets']).round(4)

# 4️⃣ Filter by minimum bets threshold
strike_df_filtered = strike_df[strike_df['total_bets'] >= MIN_STRIKE_BETS].reset_index(drop=True)

# 5️⃣ Extract Top & Bottom Strike Rate Tracks
top_strike    = strike_df_filtered.nlargest(10, 'Strike_Rate').reset_index(drop=True)
bottom_strike = strike_df_filtered.nsmallest(10, 'Strike_Rate').reset_index(drop=True)

# 6️⃣ Preview
print(f"✅ Strike rates computed (min {MIN_STRIKE_BETS} bets):")
print("Top 10 Strike Rates:")
print(top_strike[['Sport', 'Track_Name', 'total_bets', 'wins', 'Strike_Rate']])
print("\nBottom 10 Strike Rates:")
print(bottom_strike[['Sport', 'Track_Name', 'total_bets', 'wins', 'Strike_Rate']])


✅ Strike rates computed (min 50 bets):
Top 10 Strike Rates:
          Sport              Track_Name  total_bets  wins  Strike_Rate
0  Horse Racing                Rosehill          76    61       0.8026
1  Horse Racing                    York          56    43       0.7679
2  Horse Racing                 Chester          51    39       0.7647
3  Horse Racing                 Newbury          72    53       0.7361
4  Horse Racing               Ellerslie          55    40       0.7273
5  Horse Racing                Brighton          62    45       0.7258
6  Horse Racing               Chantilly          72    52       0.7222
7  Horse Racing             Musselburgh          59    42       0.7119
8  Horse Racing  Horseshoe Indianapolis          94    66       0.7021
9  Horse Racing                 Windsor          81    56       0.6914

Bottom 10 Strike Rates:
              Sport    Track_Name  total_bets  wins  Strike_Rate
0  Greyhound Racing        Hobart         139    64       0.4604
1  G

In [47]:
# --- STEP 7: Prepare all_sheets for export ---

# Core summaries
all_sheets = {
    'By Day':           by_day,
    'By Day Sorted':    by_day.sort_values('Profit_Loss', ascending=False).reset_index(drop=True),
    'By Week':          by_week,
    'Cumulative':       by_day[['Day', 'Cumulative_Profit_Loss']].rename(columns={'Cumulative_Profit_Loss': 'Cumulative'}),
    'By Month':         by_month,
    'By Sport':         by_sport,
    'By Country':       by_country,

    # Track summaries
    'Track Stats':             tracks['Track Stats'],
    'Top Horse Tracks':        tracks['Top Horse Tracks'],
    'Bottom Horse Tracks':     tracks['Bottom Horse Tracks'],
    'Top Greyhound Tracks':    tracks['Top Greyhound Tracks'],
    'Bottom Greyhound Tracks': tracks['Bottom Greyhound Tracks'],

    # Strike rates
    'Top Strike Rates':    top_strike,
    'Bottom Strike Rates': bottom_strike,
}

# Add sport-specific daily performance tables
all_sheets.update(sport_daily)

# Summary
print(f"✅ Prepared {len(all_sheets)} tables for export:")
for name in all_sheets:
    print(f"  • {name}")


✅ Prepared 28 tables for export:
  • By Day
  • By Day Sorted
  • By Week
  • Cumulative
  • By Month
  • By Sport
  • By Country
  • Track Stats
  • Top Horse Tracks
  • Bottom Horse Tracks
  • Top Greyhound Tracks
  • Bottom Greyhound Tracks
  • Top Strike Rates
  • Bottom Strike Rates
  • Snooker Daily
  • Ice Hockey Daily
  • Horse Racing Daily
  • Golf Daily
  • Politics Daily
  • Tennis Daily
  • Greyhound Racing Daily
  • Football Daily
  • Motor Sport Daily
  • Cricket Daily
  • Darts Daily
  • Basketball Daily
  • American Football Daily
  • Rugby Union Daily


In [48]:
print("📊 Top Horse Tracks preview:")
print(tracks['Top Horse Tracks'].head())
print("📊 Bottom Horse Tracks preview:")
print(tracks['Bottom Horse Tracks'].head())


📊 Top Horse Tracks preview:
            Sport Track_Name  Profit_Loss
169  Horse Racing  Geraldton      1400.78
57   Horse Racing    Aintree      1286.07
297  Horse Racing   Rosehill      1198.06
255  Horse Racing  Newcastle      1030.43
315  Horse Racing  Southwell      1026.10
📊 Bottom Horse Tracks preview:
            Sport    Track_Name  Profit_Loss
344  Horse Racing  Turfway Park      -336.93
292  Horse Racing         Ripon      -154.66
360  Horse Racing     Wincanton      -129.13
358  Horse Racing      Wetherby      -103.88
113  Horse Racing  Charles Town       -89.48


In [49]:
# --- STEP 8: Export to Google Sheets ---

import gspread
from gspread_dataframe import set_with_dataframe
from google.colab import auth
from google.auth import default
from datetime import date

# 1️⃣ Authenticate and connect
auth.authenticate_user()
creds, _ = default()
gc = gspread.authorize(creds)

# 2️⃣ Open Google Sheet
sh = next((s for s in gc.openall() if s.title == GOOGLE_SHEET_NAME), None)
if not sh:
    raise Exception(f"❌ Sheet not found: {GOOGLE_SHEET_NAME}")
print(f"✅ Connected to '{GOOGLE_SHEET_NAME}'")

# 3️⃣ Upload each table
for name, df_out in all_sheets.items():
    # Format Profit_Loss
    if 'Profit_Loss' in df_out.columns:
        df_out['Profit_Loss'] = pd.to_numeric(df_out['Profit_Loss'], errors='coerce').round(2)
        df_out['Profit_Loss'] = df_out['Profit_Loss'].map(lambda x: f"{x:.2f}" if pd.notnull(x) else "")

    # Format week date
    if 'Week Starting' in df_out.columns:
        df_out['Week Starting'] = df_out['Week Starting'].astype(str)

    # Round other numeric columns
    for col in df_out.select_dtypes(include=['float', 'int']).columns:
        df_out[col] = df_out[col].round(2)

    # Upload to sheet
    try:
        ws = sh.worksheet(name)
        ws.clear()
    except gspread.exceptions.WorksheetNotFound:
        ws = sh.add_worksheet(title=name, rows=1000, cols=20)

    set_with_dataframe(ws, df_out)
    print(f"✅ Uploaded tab: {name}")

# 4️⃣ Update KPI dashboard
try:
    dash = sh.worksheet('Dashboard')
    dash.clear()
except gspread.exceptions.WorksheetNotFound:
    dash = sh.add_worksheet('Dashboard', rows=10, cols=5)

total_profit = round(df['Profit_Loss'].sum(), 2)
total_bets   = len(df)
best_day     = df.groupby(df['Settled date'].dt.date)['Profit_Loss'].sum().idxmax()
worst_day    = df.groupby(df['Settled date'].dt.date)['Profit_Loss'].sum().idxmin()

kpis = [
    ['Metric', 'Value'],
    ['Total Profit/Loss', total_profit],
    ['Number of Bets', total_bets],
    ['Best Day', str(best_day)],
    ['Worst Day', str(worst_day)],
    ['Generated on', str(date.today())]
]
dash.update(values=kpis, range_name='A1')

print("✅ Dashboard KPIs updated")


✅ Connected to 'Betfair Dashboard'
✅ Uploaded tab: By Day
✅ Uploaded tab: By Day Sorted
✅ Uploaded tab: By Week
✅ Uploaded tab: Cumulative
✅ Uploaded tab: By Month
✅ Uploaded tab: By Sport
✅ Uploaded tab: By Country
✅ Uploaded tab: Track Stats
✅ Uploaded tab: Top Horse Tracks
✅ Uploaded tab: Bottom Horse Tracks
✅ Uploaded tab: Top Greyhound Tracks
✅ Uploaded tab: Bottom Greyhound Tracks
✅ Uploaded tab: Top Strike Rates
✅ Uploaded tab: Bottom Strike Rates
✅ Uploaded tab: Snooker Daily
✅ Uploaded tab: Ice Hockey Daily
✅ Uploaded tab: Horse Racing Daily
✅ Uploaded tab: Golf Daily
✅ Uploaded tab: Politics Daily
✅ Uploaded tab: Tennis Daily
✅ Uploaded tab: Greyhound Racing Daily
✅ Uploaded tab: Football Daily
✅ Uploaded tab: Motor Sport Daily
✅ Uploaded tab: Cricket Daily
✅ Uploaded tab: Darts Daily
✅ Uploaded tab: Basketball Daily
✅ Uploaded tab: American Football Daily
✅ Uploaded tab: Rugby Union Daily
✅ Dashboard KPIs updated
